In [1]:
##Adds the path to the codes
import os
import sys

path_common="/home/jkipen/ProtInfGPU/code/";
sys.path.append(path_common) #Adds common path to import the python files

In [2]:
from ProteinInferenceEMv2 import ProteinInferenceEMv2
import numpy as np
import pandas as pd
import time

path_5_prot="/home/jkipen/raid_storage/ProtInfGPU/data/5_Prot/"


In [3]:
PIEM=ProteinInferenceEMv2(path_5_prot+"numpy/Common/df_info.csv")



# Veryfing P(X/I) calculation
First we compute this calculation with the code that we have been using for a while, which we know we can trust

## Calc of true result


In [6]:
#Loading scores from RF_10_10 results
data = np.load(path_5_prot + "numpy/rf_n_est_10_depth_10/Common/Scores_0.npz");
top_n_flu_scores = data["top_n_flu_scores"]
top_n_flu_iz = data["top_n_flu_iz"]
#Lets keep only 1000 reads, since for higher proteins we will use a few amount of reads in each compute too
top_n_flu_scores=top_n_flu_scores[:1000,:]; 
top_n_flu_iz=top_n_flu_iz[:1000,:]; 
#Generate sparse matrix
S, p_rem = PIEM.build_sparse_matrix_N_Best(top_n_flu_scores, top_n_flu_iz, N=30) #N=30 was good for this dataset


In [9]:
n_runs=100; #Run many times to have a good time estimate.
ts=[];
for i in range(n_runs):
    t_start=time.time()
    P_X_given_I_rel = PIEM.get_P_X_given_I_rel_sparse(S, p_rem)
    ts.append(time.time()-t_start)

print("Avg runtime on CPU: " + str(np.mean(ts)));
np.save("True_P_X_given_I_rel.npy",P_X_given_I_rel)

Avg runtime on CPU: 0.0016315102577209473


## Tools to compare results
Func to test difference in values

In [17]:
def stats_diff(true_mat,comp_mat):
    diff_abs=np.abs(comp_mat-true_mat);
    mae=np.mean(diff_abs, axis = None)
    max_err=np.max(diff_abs, axis = None)
    idx_max=diff_abs.argmax();
    print("The MAE was " + str(mae) + " and the max error is  " + str(max_err) + " and was at index " + str(idx_max))
    print("The max err index had "  + str(true_mat.flatten()[idx_max])+ " in the original, but " +str(comp_mat.flatten()[idx_max]) +" in the other matrix")
    

In [18]:
random_guess=np.random.normal(size=np.shape(P_X_given_I_rel))
stats_diff(P_X_given_I_rel,random_guess)

The MAE was 0.8051492915059574 and the max error is  3.6314076041403314 and was at index 913
The max err index had 0.0009243632666766644 in the original, but 3.632331967407008 in the other matrix


Exporting the arrays needed for a cuda code to reproduce this on its own:

In [34]:
def save_array_to_csv(arr,file_name):
    df = pd.DataFrame(arr)
    df.to_csv("RepVars/"+file_name, header=False, index=False)

def export_p_flu_e_given_I(p_fe_g_I):
    n_prots=np.shape(p_fe_g_I)[0]
    n_fe=np.shape(p_fe_g_I)[1]
    p_fe_for_I_all=[]
    p_fe_for_I_iz_fe_all=[]
    n_fe_per_I=[]
    for i in range(n_prots):
        p_fe_for_I=[] #List containing the probs for a given protein'
        p_fe_for_I_iz_fe=[] #List containing the index of the flu
        for j in range(n_fe):
            if p_fe_g_I[i,j]!=0:
                p_fe_for_I.append(p_fe_g_I[i,j]);p_fe_for_I_iz_fe.append(j);
        p_fe_for_I_all.append(p_fe_for_I);
        p_fe_for_I_iz_fe_all.append(p_fe_for_I_iz_fe);
        n_fe_per_I.append(len(p_fe_for_I));
    return p_fe_for_I_all,p_fe_for_I_iz_fe_all,n_fe_per_I;



In [37]:
p_fe_for_I_all,p_fe_for_I_iz_fe_all,n_fe_per_I=export_p_flu_e_given_I(PIEM.P_flu_exp_given_prot_i)
save_array_to_csv(top_n_flu_scores[:,:30],"top_n_flu_scores.csv")
save_array_to_csv(top_n_flu_iz[:,:30],"top_n_flu_iz.csv")
save_array_to_csv(np.concatenate(p_fe_for_I_all),"p_fe_for_I_all.csv")
save_array_to_csv(np.concatenate(p_fe_for_I_iz_fe_all),"p_fe_for_I_iz_fe_all.csv")
save_array_to_csv(np.array(n_fe_per_I),"n_fe_per_I.csv")